# 4. 신경망 학습

학습은 훈련 데이터를 통해 최적의 가중치 매개변수를 자동으로 획득하는 것을 의미한다. 

## 4-1. 데이터와 데이터 주도 학습

사람이 직접 알고리즘 짜지 않고 데이터로부터 패턴을 발견하는 것이 기계 학습이다. 예컨대 특정 숫자의 특징을 이미지로부터 사람이 직접 추출하고, 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있다. 다만 이 때도 이미지를 벡터로 변환할 때 사용하는 특징은 사람이 설계(SIFT, HOG 등)해야 한다. 신경망(딥러닝)은 이와 달리 end-to-end machine learning이기 때문에 이미지에 포함된 중요한 특징까지도 기계가 스스로 학습한다.

이 때, 사용되는 데이터는 '훈련 데이터'와 '시험 데이터'로 나뉜다. 이렇게 목적에 따라 데이터를 나눔으로써 신경망의 범용적 성능을 알 수 있게 된다. 범용적 성능을 가지고 있지 않은 신경망은, 예를 들어 특정 인물의 글씨체(특정 데이터셋)만 더욱 잘 인식한다거나 하는 등의 '오버피팅(overfitting)'에 직면할 수 있다. 

## 4-2. 손실 함수

'신경망의 성능이 얼마나 나쁜가'의 척도. 신경망은 이 손실 함수를 기준으로 최적의 매개변수 값을 탐색한다. 손실함수는 일반적으로 오차제곱합과 교차 엔트로피 오차를 사용한다.

중요한 점은 신경망을 학습할 때는 정확도를 지표로 삼을 수 없다는 것이다. 정확도를 지표로 삼으면 대부분의 장소에서 매개변수(편향치와 가중치)의 미분 값이 0이 되기 때문이다.

### 4-2-1. 오차제곱합 (Sum of Squres for Error, SEE)

말 그대로 오차의 제곱 합이다. 정답 레이블을 $t$, 신경망의 출력(신경망이 추정한 값)을 $y$라 할 때, SSE는 다음과 같다.


$$\frac{1}{2}\sum_k(y_k-t_k)^2$$

예를 들어, 이미지를 입력으로 받아 숫자를 추정하는 프로그램이라고 하면, 출력은 0 ~ 9로 총 1x10 행렬일 것이다. 정답은 10개 중 하나일 것이므로 단 하나만 1 로 출력되어 '원-핫 인코딩'이 될 것이고, 신경망의 출력과 정답 레이블은 각각

```python
y = [0.1, 0.7, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0] # 신경망의 출력. 소프트맥스 함수이므로 각 숫자일 확률로 볼 수 있다.
t = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0] # 정답은 1
```
과 같이 될 것이다. 소프트맥스 함수의 출력값과 정답 레이블의 오차가 커질수록 SSE 함수의 출력도 커질 것이므로 이는 '신경망의 나쁨의 정도'로 적절히 사용될 수 있다.

### 4-2-2. 교차 엔트로피 오차 (Cross Entropy Error, CEE)

$$E = -\sum_kt_k\log y_k$$

이 때 정답 레이블 $t_k$는 오직 정답에 대해서만 1이고 나머지는 0인 원-핫 인코딩이므로, 결국 정답 레이블과 같은 인덱스의 출력 $y_k$가 1.0에 다가갈수록 0에 가까워지고, 0.0에 가까워질수록 1에 가까워져 오차의 정도를 계산할 수 있게 된다. 코드로 쓰면 아래와 같다.

In [22]:
import numpy as np

def calculate_single_cee(y : np.array, t : np.array) -> float :
    delta = 1e-7
    return  -1 * np.sum(t * np.log(y + delta))

print(calculate_single_cee(np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), np.array([0, 0, 0.5, 0.2, 0, 0, 0.1, 0.2, 0, 0])))
print(calculate_single_cee(np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), np.array([0, 0.2, 0.1, 0.2, 0, 0, 0.1, 0.2, 0.1, 0.1])))


8.059047775479163
14.50628607586249


이 때, 로그 함수는 점근선이 $x = 0$ 이므로 아주 작은 값 delta를 더하여 Nan 출력을 방지해준다.

지금까지 구한 손실함수는 데이터 1개에 대한 손실함수이므로, N 개의 데이터에 대해 교차 엔트로피 오차를 구하면, 

$$E = -\frac{1}{N}\sum_n \sum_kt_{nk} \log y_{nk}$$

로 정규화할 수 있다. 이렇게 정규화함으로써 데이터의 크기가 1000이든 100이든 통일된 오차 지표를 구할 수 있다.

그런데 보통 데이터는 매우 많으므로, 일반적으로는 랜덤으로 조금씩 빼서 손실 함수를 구하고 학습하는데, 이러한 학습 방법을 미니배치 학습이라고 한다.

In [23]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label = True)

print(x_train.shape) # 28 * 28 = 784
print(t_train.shape)

train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)
print(batch_mask, ", type : ", type(batch_mask))
x_train_batch = x_train[batch_mask]
print("x_train_batch.shape : ", x_train_batch.shape)
t_train_batch = t_train[batch_mask]
print("t_train_batch.shape : ", t_train_batch.shape)

(60000, 784)
(60000, 10)
[13411 23365  1986 18721 15774 28718 44431 24394 44785 25994] , type :  <class 'numpy.ndarray'>
x_train_batch.shape :  (10, 784)
t_train_batch.shape :  (10, 10)


이를 바탕으로 10개의 데이터에 대해 교차 엔트로피 에러를 구해보면 아래와 같다.

In [26]:
def calculate_cee(y : np.array, t : np.array) -> float :
    if y.ndim == 1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    delta = 1e-7
    return -1 * np.sum(t * np.log(y + delta)) / batch_size